# Height-based results

In [1]:
from pathlib import Path
import xarray as xr
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import numpy as np
from matplotlib.colors import ListedColormap, BoundaryNorm, LogNorm, to_hex
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
from plot_utils import PHASE_TO_NUM_MAP, PHASE_MAP, cblind_cmap, PHASE_LABEL_MAP
from tqdm.notebook import tqdm


def title(label: str) -> str:
    return label.replace("_", " ").title()

def rgb_to_hex(rgb):
    """Function to convert the sns colormap/values to hex for plotly"""
    return to_hex(rgb)

PHASE_NUMS = [1, 2, 3, 4, 5, 6, 7]  # ignore cloudy and unknown pixels for accuracy
PHASES = [PHASE_MAP[i] for i in PHASE_NUMS]

In [2]:
from typing import Any
import plotly.graph_objects as go

def create_figure(
    size: tuple[int, int] = (1200, 800),
    legend_loc: tuple[float, float] = (0.075, 1.0),
    xaxis: dict[str, Any] | None = None,
    xaxis2: dict[str, Any] | None = None,
    yaxis: dict[str, Any] | None = None,
    yaxis2: dict[str, Any] | None = None,
    legend: dict[str, Any] | None = None,
    **kwargs,
):
    """Create a plotly.graph_objects.Figure with some preferred settings.

    Args:
        size (tuple[int, int], optional): Width, height tuple. Defaults to (1200, 800).
        legend_loc (tuple[float, float], optional): Legend location. Defaults to (0.075, 1.0).
        xaxis (dict[str, Any] | None, optional): Optional extra keyword arguments for go.Layout(xaxis=...). Defaults to None.
        xaxis2 (dict[str, Any] | None, optional): Optional extra keyword arguments for go.Layout(xaxis2=...). Defaults to None.
        yaxis (dict[str, Any] | None, optional): Optional extra keyword arguments for go.Layout(yaxis=...). Defaults to None.
        yaxis2 (dict[str, Any] | None, optional): Optional extra keyword arguments for go.Layout(yaxis2=...). Defaults to None.
        legend (dict[str, Any] | None, optional): Optional extra keyword arguments for go.Layout(legend=...). Defaults to None.

    Returns:
        plotly.graph_objects.Figure: The plotly figure with specified + preferred layout settings.
    """
    xaxis = xaxis or {}
    xaxis2 = xaxis2 or {}
    yaxis = yaxis or {}
    yaxis2 = yaxis2 or {}
    legend = legend or {}
    layout_settings = dict(
        width=size[0],
        height=size[1],
        barmode="stack",
        bargap=0,
        bargroupgap=0,
        font=dict(family="serif", size=36, color="black"),
        xaxis={
            **dict(
                showline=True,
                linewidth=2,
                linecolor="black",
                ticks="outside",
                tickwidth=2,
                tickfont_size=22,
                mirror=True,
            ),
            **xaxis,
        },
        xaxis2={
            **dict(
                showline=True,
                linewidth=2,
                linecolor="black",
                ticks="outside",
                tickwidth=2,
                tickfont_size=22,
            ),
            **xaxis2,
        },
        yaxis={
            **dict(
                side="left",
                showgrid=False,
                showline=True,
                linewidth=2,
                linecolor="black",
                ticks="outside",
                tickwidth=2,
                tickfont_size=22,
                mirror=True,
            ),
            **yaxis,
        },
        yaxis2={
            **dict(
                overlaying="y",
                side="right",
                showgrid=False,
                showline=True,
                linewidth=2,
                linecolor="black",
                ticks="outside",
                tickfont_size=22,
                tickwidth=2,
            ),
            **yaxis2,
        },
        plot_bgcolor="white",
        legend={
            **dict(
                x=legend_loc[0],
                y=legend_loc[1],
                bgcolor="rgba(0,0,0,0.1)",
                orientation="h",
                traceorder="normal",
                font_size=28,
            ),
            **legend,
        },
        margin=dict(t=0, b=0, l=0, r=10)
    )
    layout_settings.update(**kwargs)
    fig = go.Figure(layout=go.Layout(**layout_settings))
    return fig

In [15]:
models = ["cnn", "mlp_balanced", "rf_balanced"]

df = pd.read_parquet("./data/parallel_nsa_cloudy_predictions.parquet")[models + ["cloud_phase"]]
df = df.rename(columns={"mlp_balanced": "mlp", "rf_balanced": "rf"})

df

cnn     mlp      rf cloud_phase
time                height                                    
2021-01-01 00:57:00 0.16       ice  liquid  liquid         ice
                    0.19       ice     ice  liquid         ice
                    0.22       ice     ice     ice         ice
                    0.25       ice     ice  liquid         ice
2021-01-01 00:57:30 0.16       ice  liquid     ice         ice
...                            ...     ...     ...         ...
2021-12-31 23:59:30 2.38    liquid     ice     ice         ice
                    2.41    liquid     ice     ice         ice
                    2.44    liquid     ice     ice         ice
                    2.47    liquid     ice     ice         ice
                    2.50    liquid     ice     ice         ice

[64930931 rows x 4 columns]

In [10]:
def iou(df: pd.DataFrame, pred: str, phases: tuple[str], thresh: int = 1) -> pd.DataFrame:
    """Calculates IOU for each phase type"""
    summary = {}
    for phase in phases:
        union = df[(df["cloud_phase"] == phase) | (df[pred] == phase)]
        if (df["cloud_phase"] == phase).sum() >= thresh:  # cutoff if not enough ground truth data
            intersection = union["cloud_phase"] == union[pred]
            summary[f"{phase}_iou"] = intersection.sum() / len(union)
    summary["mean_iou"] = np.mean(list(summary.values())) if len(summary) else None
    return pd.DataFrame({pred: summary})

def acc(df: pd.DataFrame, pred: str, phases: tuple[str], thresh: int = 1) -> pd.DataFrame:
    """Calculates accuracy for each phase type"""
    summary = {}
    for phase in phases:
        all_of_phase = df[df["cloud_phase"] == phase]
        if len(all_of_phase) >= thresh:  # cutoff if not enough ground truth data
            summary[f"{phase}_acc"] = 100 * (all_of_phase[pred] == phase).mean()
    summary["accuracy"] = 100 * (df["cloud_phase"] == df[pred]).mean()
    return pd.DataFrame({pred: summary})


def accuracy(df: pd.DataFrame, pred: str) -> float:
    return (df["cloud_phase"] == df[pred]).mean() * 100

def precision(df: pd.DataFrame, pred: str, phases: tuple[str], thresh: int = 1) -> pd.DataFrame:
    summary = {}
    for phase in phases:
        true_positives = ((df["cloud_phase"] == phase) & (df[pred] == phase)).sum()
        pred_positives = (df[pred] == phase).sum()
        if (df["cloud_phase"] == phase).sum() >= thresh:  # cutoff if not enough ground truth data
            if pred_positives > 0:
                summary[f"{phase}_precision"] = true_positives / pred_positives
    summary["mean_precision"] = np.mean(list(summary.values())) if len(summary) else None
    return pd.DataFrame({pred: summary})

def recall(df: pd.DataFrame, pred: str, phases: tuple[str], thresh: int = 1) -> pd.DataFrame:
    summary = {}
    for phase in phases:
        true_positives = ((df["cloud_phase"] == phase) & (df[pred] == phase)).sum()
        all_positives = (df["cloud_phase"] == phase).sum()
        if (df["cloud_phase"] == phase).sum() >= thresh:  # cutoff if not enough ground truth data
            if all_positives > 0:
                summary[f"{phase}_recall"] = true_positives / all_positives
    summary["mean_recall"] = np.mean(list(summary.values())) if len(summary) else None
    return pd.DataFrame({pred: summary})

def f1_from_precision_recall(precision_score, recall_score) -> float | None:
    if pd.isna(precision_score) or pd.isna(recall_score):
        return None
    if precision_score + recall_score == 0:
        return None
    return 2 * (precision_score * recall_score) / (precision_score + recall_score)

In [11]:
heights = df.reset_index()["height"].unique()

In [16]:
acc(df, "cnn", PHASES), acc(df, "mlp", PHASES)

(                    cnn
 accuracy      95.700364
 drizzle_acc   82.555176
 ice_acc       98.512772
 liq_driz_acc  81.872068
 liquid_acc    88.107325
 mixed_acc     83.710018
 rain_acc      97.557382
 snow_acc      93.796499,
                     mlp
 accuracy      84.645356
 drizzle_acc   85.263294
 ice_acc       81.811465
 liq_driz_acc  90.540118
 liquid_acc    88.573055
 mixed_acc     88.717578
 rain_acc      96.104204
 snow_acc      97.596860)

In [17]:
df["cnn"].value_counts()

ice         47070960
mixed        5602652
snow         5313096
liquid       4556647
rain         1227002
drizzle       619162
liq_driz      541412
Name: cnn, dtype: int64

In [ ]:
import sklearn.metrics as skm
from joblib import Parallel, delayed

THRESH = 1000

half_bin_width = 0.25
bin_centers = np.arange(0.0, 11, 0.5)
lower_bounds = bin_centers - half_bin_width
upper_bounds = bin_centers + half_bin_width

results = {
    "height": [],
    "model": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "iou": [],
}
models = [
    "cnn",
    "mlp",
    "rf",
]

def compute_metrics_v2(height, df_height, model):
    accuracy_score = accuracy(df_height, model)
    recall_score = recall(df_height, model, PHASES, thresh=THRESH).loc["mean_recall"].item()  # type: ignore
    # recall_score = None
    precision_score = precision(df_height, model, PHASES, thresh=THRESH).loc["mean_precision"].item()  # type: ignore
    # precision_score = None
    # f1_score = 2 * (precision_score * recall_score) / (precision_score + recall_score)
    f1_score = f1_from_precision_recall(precision_score, recall_score)
    iou_score = iou(df_height, model, PHASES, thresh=THRESH).loc["mean_iou"].item()  # type: ignore
    # f1_score = None
    # iou_score = None
    return height, model, accuracy_score, precision_score, recall_score, f1_score, iou_score

def process_interval(i, data, model):
    lower, upper = lower_bounds[i], upper_bounds[i]
    filtered_df = data[(data['height'] >= lower) & (data['height'] <= upper)]
    return compute_metrics_v2(bin_centers[i], filtered_df, model)

results_list = Parallel(n_jobs=16)(
    delayed(process_interval)(i, df.reset_index(), model)
    for i in tqdm(range(len(bin_centers)))
    for model in models
)


for height, model, accuracy_score, precision_score, recall_score, f1_score, iou_score in results_list:  # type: ignore
    results["height"].append(height)
    results["model"].append(model)
    results["accuracy"].append(accuracy_score)
    results["precision"].append(precision_score)
    results["recall"].append(recall_score)
    results["f1"].append(f1_score)
    results["iou"].append(iou_score)

metrics_df = pd.DataFrame(results)  # type: ignore
metrics_df.head(10)

  0%|          | 0/22 [00:00<?, ?it/s]

,height,model,accuracy,precision,recall,f1,iou
0,0.0,cnn,90.288293,0.870340,0.877442,0.873876,0.780343
1,0.0,mlp,69.380813,0.790134,0.840333,0.814461,0.663898
2,0.0,rf,71.492063,0.811235,0.850036,0.830182,0.690412
3,0.5,cnn,91.725504,0.883911,0.893579,0.888719,0.805074
4,0.5,mlp,74.351498,0.784832,0.851008,0.816581,0.676449
5,0.5,rf,75.252011,0.798282,0.857593,0.826875,0.692871
6,1.0,cnn,93.253292,0.912428,0.907662,0.910039,0.838283
7,1.0,mlp,76.183853,0.787179,0.870667,0.826821,0.692527
8,1.0,rf,76.584125,0.797272,0.873570,0.833679,0.703947
9,1.5,cnn,93.719954,0.892840,0.898181,0.895503,0.815189


In [22]:
counts = (
    df.reset_index()  # type: ignore
    .groupby(["height", "cloud_phase"])
    .size()
    .reset_index(name="count")
    .rename(columns={"cloud_phase": "phase"})
)
counts["percent"] = (
    counts["count"] / counts.groupby("height")["count"].transform("sum") * 100
)
counts

,height,phase,count,percent
0,0.16,liquid,111167,19.257800
1,0.16,ice,234450,40.614492
2,0.16,mixed,122302,21.186750
3,0.16,drizzle,16677,2.889008
4,0.16,liq_driz,7135,1.236018
...,...,...,...,...
2683,11.65,mixed,0,0.000000
2684,11.65,drizzle,0,0.000000
2685,11.65,liq_driz,0,0.000000
2686,11.65,rain,0,0.000000


In [23]:
count_pivot = counts.pivot(index="height", columns="phase", values="count")[
    PHASES
].reset_index()
count_pivot

phase,height,liquid,ice,mixed,drizzle,liq_driz,rain,snow
0,0.16,111167,234450,122302,16677,7135,21390,64136
1,0.19,113623,235180,122096,15269,7365,21032,62866
2,0.22,118208,230978,121208,14081,7954,20517,61544
3,0.25,121213,221319,118389,12947,9839,19855,60688
4,0.28,120777,210554,116419,12458,10615,19717,59582
...,...,...,...,...,...,...,...,...
379,11.53,0,61,0,0,0,0,0
380,11.56,0,39,0,0,0,0,0
381,11.59,0,25,0,0,0,0,0
382,11.62,0,12,0,0,0,0,0


In [36]:
size = (950, 1200)
fig = create_figure(
    size=size,
    legend_loc=(0.5, -0.13),
    xaxis=dict(title="Model Score", range=[0.0, 1.01], overlaying='x2', side="top"),
    xaxis2=dict(title="Phase Count", side="bottom"),
    yaxis=dict(title="Height (km)", range=[0.15, 10]),
    legend=dict(bgcolor=None, xanchor="center"),
)

# Plot counts
phases = {
    "ice": "ice", 
    "snow": "snow",
    "mixed": "mixed", 
    "rain": "rain", 
    "drizzle": "drizzle", 
    "liq_driz": "liq_driz", 
    "liquid": "liquid", 
    # "avg": "Avg",
}
count_traces = []
for phase, phase_label in phases.items():
    color = cblind_cmap[phase]
    if isinstance(color, tuple):
        color = rgb_to_hex(color)
    count_traces.append(
        go.Bar(
            x=count_pivot[phase],
            y=count_pivot["height"],
            orientation="h",
            marker_line_width=0,
            marker_color=color,
            name=phase_label,
            opacity=0.8,
            xaxis="x2",
        )
    )
fig.add_traces(count_traces)


# Plot metric lines
metric_traces = []
models_to_plot = {
    "rf": ("RF", "orange"),
    "mlp": ("MLP", "blue"),
    "cnn": ("CNN", "green"),
}
metrics_to_plot = {
    "iou": ("IOU", "dot"),
    "f1": ("F1-Score", "solid"),
}
for metric, (metric_label, line_style) in metrics_to_plot.items():
    for model, (model_label, color) in models_to_plot.items():
        model_score_dict = metrics_df[metrics_df["model"] == model]
        metric_traces.append(
            go.Scatter(
                x=model_score_dict[metric],
                y=model_score_dict["height"],
                line=dict(
                    color=color,
                    width=4,
                    dash=line_style,
                ),
                name=f"{model_label} {metric_label}",
            )
        )
fig.add_traces(metric_traces)
fig.write_image("figures/height_scores.png", scale=5, width=size[0], height=size[1])
fig.show()